In [5]:
import os
import random
from typing import Dict, List, Optional, Tuple
from collections import defaultdict
import json

In [6]:
class PromptGenerator:
    def __init__(self, prompt_data: Dict[str, Dict[str, List[str]]]):
        self.prompt_data = prompt_data
        # Initialize with default values if certain categories don't exist
        if "nouns" not in self.prompt_data:
            self.prompt_data["nouns"] = {"general": ["object", "person", "place"]}
        if "settings" not in self.prompt_data:
            self.prompt_data["settings"] = {"general": ["indoors", "outdoors", "fantasy world"]}
        if "styles" not in self.prompt_data:
            self.prompt_data["styles"] = {"general": ["realistic", "abstract", "minimalist"]}
        if "themes" not in self.prompt_data:
            self.prompt_data["themes"] = {"general": ["peaceful", "chaotic", "mysterious"]}
    
    def get_random_items(self, category: str, subcategory: str = None, count: int = 1) -> List[str]:
        """Get random items from a category and optional subcategory."""
        if category not in self.prompt_data:
            return []
        
        if subcategory and subcategory in self.prompt_data[category]:
            items = self.prompt_data[category][subcategory]
        else:
            # If no subcategory specified or not found, combine all subcategories
            items = []
            for sub_items in self.prompt_data[category].values():
                items.extend(sub_items)
        
        # Make sure we don't try to get more items than available
        count = min(count, len(items))
        if count == 0:
            return []
        
        return random.sample(items, count)
    
    def generate_prompt(self, 
                        # Category toggles
                        include_nouns: bool = True,
                        include_settings: bool = True, 
                        include_styles: bool = True,
                        include_themes: bool = True,
                        
                        # Quantity controls
                        noun_count: int = 1,
                        setting_min: int = 1,
                        setting_max: int = 2,
                        style_min: int = 1,
                        style_max: int = 2,
                        theme_count: int = 1,
                        
                        # Probability controls
                        setting_probability: float = 0.9,  # Chance to include settings at all
                        theme_probability: float = 0.7,    # Chance to include themes at all
                        
                        # Custom data
                        custom_categories: Optional[Dict[str, List[str]]] = None) -> Tuple[str, Dict[str, List[str]]]:

        components = {}
        prompt_parts = []
        
        # Add custom categories if provided
        working_data = self.prompt_data.copy()
        if custom_categories:
            for category, items in custom_categories.items():
                if category not in working_data:
                    working_data[category] = {"custom": items}
                else:
                    working_data[category]["custom"] = items
        
        # Add nouns
        if include_nouns and noun_count > 0:
            nouns = self.get_random_items("nouns", count=noun_count)
            if nouns:
                components["nouns"] = nouns
                prompt_parts.extend(nouns)
        
        # Add settings
        if include_settings and random.random() < setting_probability:
            # Ensure min doesn't exceed max
            actual_setting_min = min(setting_min, setting_max)
            actual_setting_max = max(setting_min, setting_max)
            
            # If min and max are the same, use that exact number
            if actual_setting_min == actual_setting_max:
                actual_setting_count = actual_setting_min
            else:
                # Otherwise randomly choose between min and max
                actual_setting_count = random.randint(actual_setting_min, actual_setting_max)
                
            settings = self.get_random_items("settings", count=actual_setting_count)
            if settings:
                components["settings"] = settings
                prompt_parts.extend(settings)
        
        # Add styles
        if include_styles:
            # Ensure min doesn't exceed max
            actual_style_min = min(style_min, style_max) 
            actual_style_max = max(style_min, style_max)
            
            # If min and max are the same, use that exact number
            if actual_style_min == actual_style_max:
                actual_style_count = actual_style_min
            else:
                # Otherwise randomly choose between min and max
                actual_style_count = random.randint(actual_style_min, actual_style_max)
                
            styles = self.get_random_items("styles", count=actual_style_count)
            if styles:
                components["styles"] = styles
                prompt_parts.extend(styles)
        
        # Add themes (with probability)
        if include_themes and random.random() < theme_probability and theme_count > 0:
            themes = self.get_random_items("themes", count=theme_count)
            if themes:
                components["themes"] = themes
                prompt_parts.extend(themes)
        
        # Shuffle the prompt parts for variety
        random.shuffle(prompt_parts)
        prompt = ", ".join(prompt_parts)
        
        return prompt, components

In [7]:
if __name__ == "__main__":
    
    # Data loader function
    def load_prompt_data(base_dir: str):
        prompt_data = defaultdict(lambda: defaultdict(list))
        def flatten_nested_data(parent_key, value, out_dict):
            if isinstance(value, dict):
                for sub_key, sub_val in value.items():
                    full_key = f"{parent_key}.{sub_key}"
                    flatten_nested_data(full_key, sub_val, out_dict)
            elif isinstance(value, list):
                out_dict[parent_key].extend(value)
        for category in os.listdir(base_dir):
            category_path = os.path.join(base_dir, category)
            if not os.path.isdir(category_path):
                continue
            for filename in os.listdir(category_path):
                if not filename.endswith(".json"):
                    continue
                file_path = os.path.join(category_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    for key, value in data.items():
                        if key in {"description", "sources"}:
                            continue
                        if isinstance(value, list):
                            prompt_data[category][key].extend(value)
                        elif isinstance(value, dict):
                            flatten_nested_data(key, value, prompt_data[category])
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
        return dict(prompt_data)
    
    # For testing, use sample data if actual data isn't available
    try:
        prompt_data = load_prompt_data("./data")
    except Exception as e:
        print(f"Using sample data due to error: {e}")
        prompt_data = {
            "nouns": {
                "general": ["castle", "dragon", "knight", "forest", "mountain", "river", "sunset", "city", "portrait"]
            },
            "settings": {
                "general": ["medieval", "futuristic", "underwater", "space", "desert", "jungle", "arctic", "cityscape"]
            },
            "styles": {
                "general": ["realistic", "abstract", "minimalist", "surreal", "vibrant", "dark", "light", "neon"]
            },
            "themes": {
                "general": ["adventure", "mystery", "romance", "horror", "fantasy", "sci-fi", "historical", "peaceful"]
            }
        }
    
    generator = PromptGenerator(prompt_data)
    
    print("Testing prompt generator with various configurations...")
    print("\n1. Default settings:")
    for i in range(3):
        prompt, components = generator.generate_prompt()
        print(f"Prompt {i+1}: {prompt}")
        print(f"Components: {components}")
    
    print("\n2. More nouns, no themes:")
    for i in range(2):
        prompt, components = generator.generate_prompt(noun_count=3, include_themes=False)
        print(f"Prompt {i+1}: {prompt}")
        print(f"Components: {components}")
    
    print("\n3. Only styles and themes:")
    for i in range(2):
        prompt, components = generator.generate_prompt(include_nouns=False, include_settings=False, style_min=2, style_max=2)
        print(f"Prompt {i+1}: {prompt}")
        print(f"Components: {components}")
    
    print("\n4. Fixed ranges (mimicking UI settings):")
    prompt, components = generator.generate_prompt(
        setting_min=1, setting_max=1,  # Always exactly 1 setting
        style_min=2, style_max=2,      # Always exactly 2 styles
        theme_probability=1.0          # Always include theme
    )
    print(f"Prompt: {prompt}")
    print(f"Components: {components}")
    
    print("\n5. With custom category:")
    custom_categories = {
        "colors": ["blue", "red", "green", "yellow", "purple"]
    }
    for i in range(2):
        prompt, components = generator.generate_prompt(custom_categories=custom_categories)
        print(f"Prompt {i+1}: {prompt}")
        print(f"Components: {components}")
    
    print("\n6. MAUI app simulation (changing parameters like UI controls):")
    # Simulate user toggling settings on UI and generating prompts
    test_configs = [
        {"name": "Default", "params": {}},
        {"name": "No Themes", "params": {"include_themes": False}},
        {"name": "Double Nouns", "params": {"noun_count": 2}},
        {"name": "Art Focus", "params": {"style_min": 2, "style_max": 2, "setting_min": 1, "setting_max": 1}},
        {"name": "Always Theme", "params": {"theme_probability": 1.0}},
    ]
    
    for config in test_configs:
        print(f"\n--- {config['name']} ---")
        prompt, components = generator.generate_prompt(**config["params"])
        print(f"Prompt: {prompt}")
        print(f"Components: {components}")

Testing prompt generator with various configurations...

1. Default settings:
Prompt 1: outdoors, peaceful, realistic, Perilla
Components: {'nouns': ['Perilla'], 'settings': ['outdoors'], 'styles': ['realistic'], 'themes': ['peaceful']}
Prompt 2: abstract, pearl necklace, fantasy world, indoors
Components: {'nouns': ['pearl necklace'], 'settings': ['indoors', 'fantasy world'], 'styles': ['abstract']}
Prompt 3: javelin, peaceful, fantasy world, minimalist
Components: {'nouns': ['javelin'], 'settings': ['fantasy world'], 'styles': ['minimalist'], 'themes': ['peaceful']}

2. More nouns, no themes:
Prompt 1: fantasy world, realistic, crossbow, dagger, goblin
Components: {'nouns': ['goblin', 'crossbow', 'dagger'], 'settings': ['fantasy world'], 'styles': ['realistic']}
Prompt 2: outdoors, greatsword, realistic, boom box, indoors, persimmon, minimalist
Components: {'nouns': ['greatsword', 'boom box', 'persimmon'], 'settings': ['indoors', 'outdoors'], 'styles': ['minimalist', 'realistic']}

3